In [11]:
# === 1. Import Required Libraries ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings("ignore")

# === 2. Load the Dataset ===
file_path = "ITSM_data.csv"  # Replace with actual path if not in same folder
df = pd.read_csv(file_path, parse_dates=['Open_Time'], dayfirst=True)

# === 3. Clean the Data ===
df = df[['Open_Time', 'Category']].copy()  # Keep only required columns
df.dropna(subset=['Open_Time', 'Category'], inplace=True)  # Remove missing values
df['Category'] = df['Category'].astype(str).str.strip().replace('', 'Unknown')
df['Open_Time'] = pd.to_datetime(df['Open_Time'], errors='coerce')
df.dropna(subset=['Open_Time'], inplace=True)

# === 4. Time Feature Engineering ===
df['Year'] = df['Open_Time'].dt.year
df['Quarter'] = df['Open_Time'].dt.to_period('Q').astype(str)

# === 5. Aggregate Ticket Counts per Quarter per Category ===
grouped = df.groupby(['Quarter', 'Category']).size().reset_index(name='Ticket_Count')

# === 6. Convert 'Quarter' to Start-of-Quarter Timestamp ===
def quarter_str_to_date(q_str):
    year = int(q_str[:4])
    quarter = int(q_str[-1])
    month = 3 * (quarter - 1) + 1
    return pd.Timestamp(year=year, month=month, day=1)

grouped['Quarter_Start'] = grouped['Quarter'].apply(quarter_str_to_date)
grouped = grouped.sort_values(by='Quarter_Start')

# === 7. Find a Category with Sufficient Data (≥8 Quarters) ===
valid_categories = []

for category in grouped['Category'].unique():
    cat_df = grouped[grouped['Category'] == category][['Quarter_Start', 'Ticket_Count']]
    cat_ts = cat_df.set_index('Quarter_Start').asfreq('Q').dropna()
    if len(cat_ts) >= 8:
        valid_categories.append(category)
        break  # Pick the first valid category

# === 8. Proceed Only If Valid Category Found ===
if valid_categories:
    selected_category = valid_categories[0]
    cat_df = grouped[grouped['Category'] == selected_category][['Quarter_Start', 'Ticket_Count']]
    cat_ts = cat_df.set_index('Quarter_Start').asfreq('Q').dropna()

    # === A. Fit SARIMA Model ===
    model = SARIMAX(cat_ts, order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
    result = model.fit()

    # === B. Forecast Next 4 Quarters ===
    forecast_steps = 4
    future_index = pd.date_range(start=cat_ts.index[-1] + relativedelta(months=3),
                                 periods=forecast_steps, freq='Q')
    forecast = result.get_forecast(steps=forecast_steps).predicted_mean
    forecast.index = future_index

    # === C. Plot Actual vs Forecast ===
    plt.figure(figsize=(12, 6))
    plt.plot(cat_ts.index, cat_ts['Ticket_Count'], label='Actual', linewidth=2)
    plt.plot(forecast.index, forecast, label='Forecast', color='red', linestyle='--', linewidth=2)
    plt.title(f"SARIMA Forecast for Category: {selected_category}", fontsize=14)
    plt.xlabel("Quarter")
    plt.ylabel("Ticket Count")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # === D. Print Forecasted Ticket Counts ===
    forecast_rounded = forecast.round(0)
    forecast_df = forecast_rounded.reset_index()
    forecast_df.columns = ['Quarter', 'Forecast_Tickets']

    print(f"\n✅ Forecasted Ticket Volumes for Category: {selected_category}")
    print(forecast_df.to_string(index=False))

else:
    print("❌ No category found with at least 8 quarters of data.")# === 1. Import Required Libraries ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings("ignore")

# === 2. Load the Dataset ===
file_path = "ITSM_data.csv"  # Change path if running in a different directory
df = pd.read_csv(file_path, parse_dates=['Open_Time'], dayfirst=True)

# === 3. Data Cleaning ===
df = df[['Open_Time', 'Category']].copy()
df.dropna(subset=['Open_Time', 'Category'], inplace=True)
df['Category'] = df['Category'].astype(str).str.strip().replace('', 'Unknown')
df['Open_Time'] = pd.to_datetime(df['Open_Time'], errors='coerce')
df.dropna(subset=['Open_Time'], inplace=True)

# === 4. Time Feature Engineering ===
df['Year'] = df['Open_Time'].dt.year
df['Quarter'] = df['Open_Time'].dt.to_period('Q').astype(str)

# === 5. Group by Quarter and Category ===
grouped = df.groupby(['Quarter', 'Category']).size().reset_index(name='Ticket_Count')

# === 6. Convert 'Quarter' to Timestamp (start of quarter) ===
def quarter_str_to_date(q_str):
    year = int(q_str[:4])
    quarter = int(q_str[-1])
    month = 3 * (quarter - 1) + 1
    return pd.Timestamp(year=year, month=month, day=1)

grouped['Quarter_Start'] = grouped['Quarter'].apply(quarter_str_to_date)
grouped = grouped.sort_values(by='Quarter_Start')

# === 7. Identify a Category with At Least 8 Quarters of Data ===
valid_categories = []

for category in grouped['Category'].unique():
    cat_df = grouped[grouped['Category'] == category][['Quarter_Start', 'Ticket_Count']]
    cat_ts = cat_df.set_index('Quarter_Start').asfreq('Q').dropna()
    if len(cat_ts) >= 8:
        valid_categories.append(category)
        break

# === 8. Forecast if Valid Category Found ===
if valid_categories:
    selected_category = valid_categories[0]
    print(f"\n✅ Using category: {selected_category} (with ≥ 8 quarters of data)")

    # Prepare time series
    cat_df = grouped[grouped['Category'] == selected_category][['Quarter_Start', 'Ticket_Count']]
    cat_ts = cat_df.set_index('Quarter_Start').asfreq('Q').dropna()

    # === A. Fit SARIMA Model ===
    model = SARIMAX(cat_ts, order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
    result = model.fit()

    # === B. Forecast Next 4 Quarters ===
    future_index = pd.date_range(start=cat_ts.index[-1] + relativedelta(months=3),
                                 periods=4, freq='Q')
    forecast = result.get_forecast(steps=4).predicted_mean
    forecast.index = future_index

    # === C. Plot Actual vs Forecast ===
    plt.figure(figsize=(12, 6))
    plt.plot(cat_ts.index, cat_ts['Ticket_Count'], label='Actual', linewidth=2)
    plt.plot(forecast.index, forecast, label='Forecast', color='red', linestyle='--', linewidth=2)
    plt.title(f"SARIMA Forecast - {selected_category}", fontsize=14)
    plt.xlabel("Quarter")
    plt.ylabel("Ticket Count")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # === D. Print Forecasted Values ===
    forecast_rounded = forecast.round(0)
    forecast_df = forecast_rounded.reset_index()
    forecast_df.columns = ['Quarter', 'Forecast_Tickets']
    print(forecast_df.to_string(index=False))

else:
    print("❌ No category found with at least 8 quarters of data for forecasting.")



❌ No category found with at least 8 quarters of data.
❌ No category found with at least 8 quarters of data for forecasting.
